In [33]:
import os

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import norm

import matplotlib.pyplot as plt

In [34]:
downweight_granularity=['FltNum','dtime','Direction','dday','month']

# Data Import

In [35]:
#Data=pd.read_csv('~/Data/Intermediate_Output/R_Output_Test_Pax.csv',sep=',')
Data=pd.read_csv('R_Output_Test_Pax.csv',sep=',')
Data=Data[['DepDate','FltNum','yday','dtime','Direction','month','dday','Dprio','NumPax']]

Map_Dates=DataFrame([Data['DepDate'].unique().tolist(),[pd.to_datetime(x) for x in Data['DepDate'].unique().tolist()]]).transpose()
Map_Dates.columns=['DepDate','DepDate_dt']
Map_Dates['DepDate']=Map_Dates['DepDate'].astype('str')

Data=Data.merge(Map_Dates,on=['DepDate'])
Data['DepDate']=Data['DepDate_dt']
Data=Data[[x for x in Data.columns if x!='DepDate_dt']]

Data['TicketDate']=Data['DepDate']-Data['Dprio'].apply(lambda x: datetime.timedelta(x))

Data=Data.loc[Data['TicketDate']<=pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),:]

Data=Data[[x for x in Data.columns if x!='TicketDate']]

Data['Dprio']=-Data['Dprio']
Data=Data.set_index(['DepDate','FltNum','yday','dtime','Direction','month','dday','Dprio'])
Data=Data.groupby(level=[0,1,2,3,4,5,6,7]).sum().groupby(level=[0,1,2,3,4,5,6])['NumPax'].cumsum().reset_index()

Data['Dprio']=-Data['Dprio']

In [13]:
#IdealCurve=pd.read_csv('~/Data/Intermediate_Output/IdealCurve.csv')
IdealCurve=pd.read_csv('IdealCurve.csv')

In [14]:
Dropdown_comb=Data[['DepDate']+downweight_granularity].drop_duplicates()
Dropdown_comb.index=range(Dropdown_comb.shape[0])
Dprio_all=Data['Dprio'].unique().tolist()

Dropdown_comb=Dropdown_comb.loc[np.repeat(Dropdown_comb.index,len(Dprio_all)),:]
Dropdown_comb.index=range(Dropdown_comb.shape[0])

Dropdown_comb=pd.concat([Dropdown_comb,Series(Dprio_all*Dropdown_comb.drop_duplicates().shape[0])],axis=1)
Dropdown_comb=Dropdown_comb.rename(columns={0: 'Dprio'})

Dropdown_comb=Dropdown_comb.merge(IdealCurve,on=downweight_granularity+['Dprio'])

In [15]:
Dropdown_comb=Dropdown_comb.rename(columns={'Ideal_lower': 'Ramp-up frontier',
'Ideal_average_downweighted': 'Ideal curve (80% LF)', 'Ideal_average_full': 'Ideal curve (100% LF)',
'Ideal_upper': 'Phase-down frontier'})

#Dropdown_comb.to_csv('~/Data/FrontEnd_Input/IdealCurve_forPlots.csv',index=False)
Dropdown_comb.to_csv('IdealCurve_forPlots.csv',index=False)

In [16]:
#Dropdown_comb.loc[Dropdown_comb['DepDate']=='2019-06-29',:]

In [17]:
# Data=Data[['DepDate','dtime','Direction','dday','Dprio','NumPax','Ideal_lower','Ideal_average_downweighted','Ideal_average_full','Ideal_upper']]\
# .sort_values(by=['DepDate','Direction','dtime','Dprio'])

In [18]:
Data['NumPax']=Data['NumPax'].apply(lambda x: int(round(x)))

In [19]:
deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in Data['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([Data['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

Data=Data.merge(Map_DepTime,on='dtime')

Data['dtime']=Data['deptime']
Data=Data[[x for x in Data.columns if x!='deptime']]

In [20]:
Data=Data.sort_values(by=['DepDate','dtime'])
Data=Data.rename(columns={'NumPax': 'Actual Bookings'})

In [21]:
#GroupPax=pd.read_csv('~/Data/Intermediate_Output/GroupPax.csv').rename(columns={'Group pax': 'Group Pax'})
GroupPax=pd.read_csv('GroupPax.csv').rename(columns={'Group pax': 'Group Pax'})

GroupPax['DepDate']=pd.to_datetime(GroupPax['DepDate'])
GroupPax['dtime']=GroupPax['dtime'].astype('str')
GroupPax['Dprio']=GroupPax['Dprio'].astype('float')

Data=Data.merge(GroupPax[['DepDate','dtime','Dprio','Group Pax']],on=['DepDate','dtime','Dprio'])

In [22]:
# Capacity=pd.read_csv(data_folder+'/Capacity.csv')

# Capacity['DepDate']=pd.to_datetime(Capacity['DepDate'])
# Capacity['dtime']=Capacity['dtime'].astype('str')
# Capacity['Dprio']=Capacity['Dprio'].astype('float')

# Data=Data.merge(Capacity[['DepDate','dtime','Dprio','Cabin Capacity']],on=['DepDate','dtime','Dprio'])

In [23]:
#Data=Data.to_csv('~/Data/FrontEnd_Input/GUI_in.csv',index=False)
Data=Data.to_csv('GUI_in.csv',index=False)